In [1]:
#ライブラリを自動リロード
%load_ext autoreload
%autoreload 2

from LLMSearch.Siamese.DictEmbedder import DictEmbedder
from tqdm import tqdm
import json
import os
from IPython.display import clear_output
os.environ['RWKV_JIT_ON'] = '0'
os.environ["RWKV_CUDA_ON"] = '1'

from LLMSearch.Embedding.VicunaEmbedding import VicunaEmbedding
from LLMSearch.Embedding.RWKVEmbedding import RWKVEmbedding
embedder=RWKVEmbedding(model_path="/media/kh/python/2023/RWKV-4b-Pile-3B-20230228-7963.pth",
                strategy="cuda fp16"
                     )

#notebookの出力をクリア
clear_output()
dict_embedder=DictEmbedder("develop/databricks-dolly-15k-ja-vec.bin.backup",embedder,init=False)
bricks_dict=json.load(open("develop/databricks-dolly-15k-ja.json"))
#

In [2]:
import pandas as pd
q_list=[]
a_list=[]
for brick in tqdm(bricks_dict):
    q=brick["instruction"]
    a=brick["output"]
    if q in dict_embedder.dict:
        if a in dict_embedder.dict:
            q_list.append(q)
            a_list.append(a)


qa_df=pd.DataFrame({"Q":q_list,"A":a_list})
qa_df

100%|██████████| 15015/15015 [00:00<00:00, 526875.89it/s]


,Q,A
0,ヴァージン・オーストラリア航空はいつから運航を開始したのですか？,ヴァージン・オーストラリア航空は、2000年8月31日にヴァージン・ブルー航空として、2機の...
1,魚の種類はどっち？イコクエイラクブカとロープ,イコクエイラクブカ
2,ラクダはなぜ水なしで長く生きられるのか？,ラクダは、長時間にわたってエネルギーと水分で満たされた状態を保つために、腰の脂肪を利用しています。
3,アリスの両親には3人の娘がいる：エイミー、ジェシー、そして三女の名前は？,三女の名前はアリス
4,小森田友明はいつ生まれたの？,小森田友明は1981年7月10日に生まれました。
...,...,...
14781,ジオメトリーとは？,地形学（geomorphometry）とは、地形の特性、地球の表面の形状、およびこの表面形状...
14782,"troll the respawn, Jeremyとはどういう意味ですか？",Trolling the respawnとは、ビデオゲームにおいて、対戦相手がリスポーンする...
14783,2021年、チェスの世界選手権で優勝したのは？,マグナス・カールセンがイアン・ネポムニアッキを7.5 - 3.5で破り、2021年チェス世界...
14784,なぜ毎日歩くといいのでしょうか？,ウォーキングは、関節に負担をかけずにカロリーを消費できる良い運動です。年齢を問わず、1日に数...


In [118]:

import random
q_to_a_dict={}
for i in range(len(qa_df)):
    q=qa_df["Q"][i]
    a=qa_df["A"][i]
    q_to_a_dict[q]=a

q_list=list(qa_df["Q"])
a_list=list(qa_df["A"])


q_a_dataset=[]
#exact answer
for q in q_list:
    q_a_dataset.append((q,q_to_a_dict[q],1))

#random ans
for i in range(len(q_list)):
    q=random.choice(q_list)
    a=random.choice(a_list)
    if q_to_a_dict[q]!=a:
        q_a_dataset.append((q,a,-1))

In [34]:
from MeCab import Tagger
tagger=Tagger("-Owakati")

tokenized_q_list=[tagger.parse(q).strip().split(" ") for q in q_list]
tokenized_a_list=[tagger.parse(a).strip().split(" ") for a in a_list]

In [70]:
from rank_bm25 import BM25Okapi, BM25Plus, BM25L

bm25 = BM25Plus(tokenized_a_list)
#bm25 = BM25Okapi(tokenized_a_list)

n_sampling=500
bm_dataset=[]

def prepare_bm25_dataset(q_id):
    tokenized_query=tokenized_q_list[q_id]
    doc_scores = bm25.get_scores(tokenized_query)
    max_score=max(doc_scores)
    min_score=min(doc_scores)
    if max_score==min_score:
        return []

    #-1から1に正規化
    doc_scores=[(score-min_score)/(max_score-min_score)*2-1 for score in doc_scores]

    temp_dataset=[]
    for score,a in zip(doc_scores,a_list):
        temp_dataset.append((q_list[q_id],a,score))

    #ランダムにn_sampling個サンプリング
    #temp_dataset=random.sample(temp_dataset,n_sampling)

    #doc_scoresをソート
    temp_dataset=sorted(temp_dataset,key=lambda x:x[2],reverse=True)
    temp_dataset=(temp_dataset[::n_sampling])
    return temp_dataset
#for q_id in tqdm(range(len(q_list))):
#    bm_dataset.extend(temp_dataset)

from multiprocessing import Pool
with Pool(16) as p:
    bm_dataset_part=p.map(prepare_bm25_dataset,range(len(q_list)))

for temp_dataset in bm_dataset_part:
    bm_dataset.extend(temp_dataset)

In [117]:
q_a_dataset.extend(bm_dataset)
len(q_a_dataset)

472880

In [103]:
#a_listをdoc_scoresでソート
#sorted_a_list=[a for _,a in sorted(zip(doc_scores,a_list),reverse=True)]
#sorted_a_list[:10],a_list[1]
q_a_dataset[:10]

[('ヴァージン・オーストラリア航空はいつから運航を開始したのですか？',
  'ヴァージン・オーストラリア航空は、2000年8月31日にヴァージン・ブルー航空として、2機の航空機で単一路線の運航を開始しました。',
  1),
 ('魚の種類はどっち？イコクエイラクブカとロープ', 'イコクエイラクブカ', 1),
 ('ラクダはなぜ水なしで長く生きられるのか？',
  'ラクダは、長時間にわたってエネルギーと水分で満たされた状態を保つために、腰の脂肪を利用しています。',
  1),
 ('アリスの両親には3人の娘がいる：エイミー、ジェシー、そして三女の名前は？', '三女の名前はアリス', 1),
 ('小森田友明はいつ生まれたの？', '小森田友明は1981年7月10日に生まれました。', 1),
 ('ステイルメイトの時に、私の方が多くの駒を持っていたら、私の勝ちですか？',
  'いいえ。\nステイルメイトとは、引き分けた状態のことです。どちらがより多くの駒を捕獲したか、または優勢であるかは関係ない',
  1),
 ('ロラパルーザについての参考文章が与えられ、どこで行われ、誰が始めたのか、何なのか？',
  'ロラパルーズは、イリノイ州シカゴのグラントパークで毎年開催される音楽フェスティバルである。1991年、ジェーンズ・アディクションの歌手であるペリー・ファレルのお別れツアーとしてスタートした。このフェスティバルには、オルタナティブ・ロック、ヘビーメタル、パンクロック、ヒップホップ、エレクトロニック・ダンス・ミュージックなど、さまざまなジャンルの音楽が含まれています。毎年40万人の観客を動員し、チケットは毎年完売しています。主な出演者は、レッド・ホット・チリ・ペッパーズ、チャンス・ザ・ラッパー、メタリカ、レディ・ゲイジなどです。ロラパルーザは、世界最大かつ最も象徴的な音楽フェスティバルの1つであり、シカゴの定番イベントです。',
  1),
 ('国連に本部を建てるためにニューヨークの土地を寄付したのは誰？', 'ジョン・デイヴィソン・ロックフェラー', 1),
 ('スマホが人間に悪い理由', '常にスマホを操作しているのが良くないため。', 1),
 ('ジョン・モーゼス・ブローニングとは何者ですか？',
  'ジョン・モーゼス・ブローニングは

In [104]:
import numpy as np
def text_to_vec(text1,text2,dict_embedder):
    q=dict_embedder(text1)
    a=dict_embedder(text2)
    #結合
    res=np.concatenate([q,a],axis=0)
    return res


In [119]:
from sklearn.model_selection import train_test_split
#q_a_datasetをtrainとtestに分割
train_set,test_set=train_test_split(q_a_dataset,test_size=0.1,random_state=42)

train_X=[text_to_vec(text1,text2,dict_embedder) for text1,text2,_ in train_set]
train_y=[label for _,_,label in train_set]
test_X=[text_to_vec(text1,text2,dict_embedder) for text1,text2,_ in test_set]
test_y=[label for _,_,label in test_set]

In [120]:
len(train_X)

26613

In [121]:
#logistic regression
from sklearn.linear_model import LogisticRegression
#clf = LogisticRegression(random_state=0).fit(train_X, train_y)
#clf.score(train_X,train_y),clf.score(test_X,test_y)

In [122]:
"""
target_qa=test_set[0]
target_question=target_qa[0]
target_answer=q_to_a_dict[target_question]

compare_set=[(target_question,text) for _,text,_ in test_set]
cand_ans_set=[text for _,text,_ in test_set]
compare_X=[text_to_vec(text1,text2,dict_embedder) for text1,text2 in compare_set]
pred_y=clf.predict_proba(compare_X)[:,0]


#cand_ans setをpred_yの値でソート
cand_ans_set=[cand_ans for _,cand_ans in sorted(zip(pred_y,cand_ans_set),reverse=True)]
print("Q ",target_question)
print("A ",target_answer)
for i in range(10):
    print(i,cand_ans_set[i])
"""

'\ntarget_qa=test_set[0]\ntarget_question=target_qa[0]\ntarget_answer=q_to_a_dict[target_question]\n\ncompare_set=[(target_question,text) for _,text,_ in test_set]\ncand_ans_set=[text for _,text,_ in test_set]\ncompare_X=[text_to_vec(text1,text2,dict_embedder) for text1,text2 in compare_set]\npred_y=clf.predict_proba(compare_X)[:,0]\n\n\n#cand_ans setをpred_yの値でソート\ncand_ans_set=[cand_ans for _,cand_ans in sorted(zip(pred_y,cand_ans_set),reverse=True)]\nprint("Q ",target_question)\nprint("A ",target_answer)\nfor i in range(10):\n    print(i,cand_ans_set[i])\n'

In [128]:
from LLMSearch.Siamese.SiameseComparer import SiameseComparer

data1=[text1 for text1,text2,_ in train_set]
data2=[text2 for text1,text2,_ in train_set]
label_list=[label for _,_,label in train_set]

#label setの0を-1に変換
#label_list=[-1 if label==0 else label for label in label_list]

comparer=SiameseComparer(dict_embedder)
comparer.prepare_model(data1,data2,label_list,hidden_dim=256)

In [127]:
target_q="猫の名前は?"
#target_q="i am a cat"
#target_q="アセトンはどのような溶媒か?"
target_q="非プロトン性溶媒に使われるものは?"
#target_q="貴方は動物ですか?"
#target_q="貴方は猫ですか?"
#target_q="スマホが良くない理由は？"
answers=[
"吾輩は猫である",
"汝の名は女だ",
"I am a cat",
"ピペットで試薬を三回分取した",
"アセトンは有機溶媒である",
"非プロトン性溶媒として､THF､トルエンなどが挙げられる",
"トルエンをこぼしてしまいました",
"私は猫です",
"私は犬です",
"日本的霊性は，鎌倉時代に禅と浄土系思想によって初めて明白に顕現し，その霊性的自覚が現在に及ぶと述べる．",
 '常にスマホを操作しているのが良くないため。'
]

sim=comparer.compare(
    [target_q]*len(answers),
    answers)

#simをソート
answers=[(s,answer) for s,answer in sorted(zip(sim,answers),reverse=True)]
for i,(s,answer) in enumerate(answers):
    print(i,s,answer)
#comparer.compare(q,a)

0 0.9999658 私は猫です
1 0.9999657 私は犬です
2 0.99992776 トルエンをこぼしてしまいました
3 0.9998509 吾輩は猫である
4 0.99965763 アセトンは有機溶媒である
5 0.9947778 常にスマホを操作しているのが良くないため。
6 0.991239 I am a cat
7 0.24160998 ピペットで試薬を三回分取した
8 0.054823577 汝の名は女だ
9 0.047301482 日本的霊性は，鎌倉時代に禅と浄土系思想によって初めて明白に顕現し，その霊性的自覚が現在に及ぶと述べる．
10 0.039546873 非プロトン性溶媒として､THF､トルエンなどが挙げられる
